In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
cmap = cm.get_cmap('RdYlGn_r')
import os

/var/folders/9x/4jq4r89s0yl_53d6vmg31bcr0000gn/T/ipykernel_1520/3172015771.py:4: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('RdYlGn_r')


In [2]:
age_counts = {12: 5, 13: 5, 14: 5, 15: 5, 16: 5, 17: 5, 18: 5}
binge_percentages = {12: 0.8, 13: 2.5, 14: 5.5, 15: 13, 16: 37, 17: 50, 18: 88}
num_epochs = 1000
# special_event_probability = 0.07
fixed_drinker_counts = {12: 3, 13: 3, 14: 3, 15: 3, 16: 4, 17: 4, 18: 5}

drinkers = {}
ping_tracker = {age: np.zeros((age_counts[age], num_epochs), dtype=int) for age in age_counts}
players_pinged = {age: np.zeros(num_epochs, dtype=int) for age in age_counts}

def gaussian_pings(mean_percentage, num_drinkers, num_epochs):
    mean_pings = mean_percentage / 100 * num_epochs * num_drinkers
    std_dev = max(1, mean_pings * 0.12)
    raw_pings = np.random.normal(mean_pings / num_epochs, std_dev / num_epochs, num_epochs).astype(int)
    raw_pings = np.clip(raw_pings, 0, num_drinkers)
    
    total_pings = raw_pings.sum()
    target_total = int(mean_pings)
    adjustment = target_total - total_pings
    
    while adjustment != 0:
        epoch_indices = np.random.choice(num_epochs, abs(adjustment), replace=True)
        for idx in epoch_indices:
            if adjustment > 0 and raw_pings[idx] < num_drinkers:
                raw_pings[idx] += 1
                adjustment -= 1
            elif adjustment < 0 and raw_pings[idx] > 0:
                raw_pings[idx] -= 1
                adjustment += 1
            if adjustment == 0:
                break
    
    return raw_pings

for age, count in age_counts.items():
    num_drinkers = fixed_drinker_counts[age]
    drinkers[age] = np.zeros(count, dtype=bool)
    drinkers[age][:num_drinkers] = True
    np.random.shuffle(drinkers[age])

for age in age_counts:
    ping_distribution = gaussian_pings(binge_percentages[age], fixed_drinker_counts[age], num_epochs)
    drinker_indices = np.where(drinkers[age])[0]
    for epoch, count in enumerate(ping_distribution):
        if count > 0:
            selected = np.random.choice(drinker_indices, count, replace=False)
            players_pinged[age][epoch] = count
            for player in selected:
                ping_tracker[age][player, epoch] += 1

# uitkomst

normalized_ping_percentage = {}
for age in age_counts:
    total_pings = np.sum(players_pinged[age])
    total_possible_pings = num_epochs * fixed_drinker_counts[age]
    normalized_ping_percentage[age] = (total_pings / total_possible_pings) * 100 if total_possible_pings > 0 else 0

for age in age_counts:
    print(f"Age {age}: {normalized_ping_percentage[age]:.2f}% pings (target: {binge_percentages[age]}%)")
    for player in range(age_counts[age]):
        total_pings = np.sum(ping_tracker[age][player, :])
        min_pings = np.min(ping_tracker[age][player, :])
        max_pings = np.max(ping_tracker[age][player, :])
        print(f"  Player {player} pinged {total_pings} times over {num_epochs} epochs. Min ping: {min_pings}, Max ping: {max_pings}")


Age 12: 0.80% pings (target: 0.8%)
  Player 0 pinged 4 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 1 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 2 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 3 pinged 9 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 4 pinged 11 times over 1000 epochs. Min ping: 0, Max ping: 1
Age 13: 2.50% pings (target: 2.5%)
  Player 0 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 1 pinged 18 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 2 pinged 32 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 3 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 4 pinged 25 times over 1000 epochs. Min ping: 0, Max ping: 1
Age 14: 5.50% pings (target: 5.5%)
  Player 0 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 1 pinged 52 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 2 pinged 53 times over 1000 epochs. Min ping: 0, Max 

In [15]:
plt.figure(figsize=(20, 10))
n_ages = len(age_counts)
cmap = cm.get_cmap('RdYlGn_r')
for i, (age, player_count) in enumerate(age_counts.items(), 1):
    ax = plt.subplot(1, n_ages, i)
    pinged_counts = players_pinged[age]
    norm = plt.Normalize(vmin=0, vmax=player_count)
    scatter = ax.scatter(pinged_counts, np.arange(num_epochs), 
                         c=pinged_counts, cmap=cmap, norm=norm, 
                         s=10, alpha=0.7)
    ax.set_title(f"{age} jaar")
    ax.set_xlabel("Aantal agents gepingt")
    ax.set_ylabel("Generaties")
    ax.set_xlim(-1, player_count + 1)
    ax.set_ylim(-50, num_epochs + 50)
    ax.set_yticks(np.arange(0, num_epochs + 1, 100))
    ax.set_xticks([0, 1, 2, 3, 4, 5])
    ax.grid(True)
plt.tight_layout()
# os.makedirs(output_dir, exist_ok=True)
filename = "Aantal agent per eeftijd die willekeurig pingen over 1000 tests.png"
plt.savefig(f"{filename}", dpi=300, bbox_inches='tight')
plt.close()
print(f"opgeslagen")

/var/folders/9x/4jq4r89s0yl_53d6vmg31bcr0000gn/T/ipykernel_1201/3872561746.py:3: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('RdYlGn_r')


opgeslagen


In [37]:
je moet deze hele code verbeteren want de uitkomst is altijd hetzelfde. terwijl als ik de code opnieuw zou draaien, zou ik steeds een diverse uitkomst moeten krijgen.
het moet de target percentages per leeftijd zo dicht mogleijk berijken. verbeter de code door te kijken naar de random number genearator in combinatie met de gaussian distribution

look in this code its mathimatecily correct. so use the math of this and add it to your last improvement.
op deze print statement moet je de gemiddelde uitkomst vergelijken met de target uitkomst 
Leeftijd 18: 88% doel voor bingedrinken

SyntaxError: invalid syntax (445670268.py, line 1)

In [130]:
# Aantal spelers per leeftijd
age_counts = {12: 5, 13: 5, 14: 5, 15: 5, 16: 5, 17: 5, 18: 5}

# Percentage bingedrinkers per leeftijd
binge_percentages = {12: 0.8, 13: 2.5, 14: 5.5, 15: 13, 16: 37, 17: 50, 18: 88}

# Aantal ronden dat gesimuleerd wordt
num_epochs = 1000

# Hoeveel vaste drinkers per leeftijd
fixed_drinker_counts = {12: 3, 13: 3, 14: 3, 15: 3, 16: 4, 17: 4, 18: 5}

# Hier slaan we per leeftijd op wie drinker is
drinkers = {}

# Hier slaan we per speler de weerstand tegen drinken op
drink_resistance = {}

# Houdt per leeftijd en speler bij in welke epoch er is gedronken
ping_tracker = {age: np.zeros((age_counts[age], num_epochs), dtype=int) for age in age_counts}

# Houdt bij hoeveel spelers per leeftijd per epoch drinken
players_pinged = {age: np.zeros(num_epochs, dtype=int) for age in age_counts}

# Deze functie maakt een verdeling van het aantal pings per epoch
def gaussian_pings(mean_percentage, num_drinkers, num_epochs):
    # Voeg ruis toe aan het gemiddelde percentage (max ±3%)
    mean_percentage += np.random.normal(0, 1.5)
    mean_percentage = max(0, min(mean_percentage, 100))

    mean_pings = mean_percentage / 100 * num_epochs * num_drinkers
    std_dev = max(1, mean_pings * np.random.uniform(0.08, 0.15))  # Meer variatie in std

    raw_pings = np.random.normal(mean_pings / num_epochs, std_dev / num_epochs, num_epochs).astype(int)
    raw_pings = np.clip(raw_pings, 0, num_drinkers)

    # Maximaal 80% van de pings bijstellen, om variatie te behouden
    total_pings = raw_pings.sum()
    target_total = int(mean_pings)
    adjustment = int((target_total - total_pings) * 0.8)  # Niet alles corrigeren

    while adjustment != 0:
        idx = np.random.randint(0, num_epochs)
        if adjustment > 0 and raw_pings[idx] < num_drinkers:
            raw_pings[idx] += 1
            adjustment -= 1
        elif adjustment < 0 and raw_pings[idx] > 0:
            raw_pings[idx] -= 1
            adjustment += 1

    return raw_pings


# Hier wijzen we aan wie drinker is, en geven we weerstand aan de vaste drinkers
for age, count in age_counts.items():
    num_drinkers = fixed_drinker_counts[age]

    # Zet eerst alle spelers op 'geen drinker', daarna wijzen we drinkers aan
    drinkers[age] = np.zeros(count, dtype=bool)
    drinkers[age][:num_drinkers] = True
    np.random.shuffle(drinkers[age])  # Willekeurige verdeling van drinkers

    # Zet voor alle spelers de weerstand op nul (niet-drinkers negeren we straks)
    drink_resistance[age] = np.zeros(count)

    # Verdeel de weerstand gelijkmatig tussen 0.1 en 0.9 voor de vaste drinkers
    resistances = np.random.uniform(0.05, 0.95, size=num_drinkers)
    np.random.shuffle(resistances)  # Willekeurige volgorde voor variatie

    index = 0
    for i in range(count):
        if drinkers[age][i]:
            drink_resistance[age][i] = resistances[index]
            index += 1

print(drink_resistance, "\n")

# Simuleer per leeftijd wie wanneer drinkt, op basis van weerstand
for age in age_counts:
    ping_distribution = gaussian_pings(binge_percentages[age], fixed_drinker_counts[age], num_epochs)

    # Vind de indices van de spelers die drinkers zijn
    drinker_indices = np.where(drinkers[age])[0]

    for epoch, count in enumerate(ping_distribution):
        if count > 0:
            # Spelers met lage weerstand hebben meer kans om te drinken
            weights = np.array([1.0 - drink_resistance[age][i] for i in drinker_indices])
            weights /= weights.sum()  # Normaliseer de kansen

            # Kies willekeurig wie er drinkt op basis van de gewichten
            selected = np.random.choice(drinker_indices, count, replace=False, p=weights)

            players_pinged[age][epoch] = count
            for player in selected:
                ping_tracker[age][player, epoch] += 1

# Resultaten tonen: hoeveel heeft elke speler gedronken
for age in age_counts:
    total_pings_all_players = np.sum(ping_tracker[age])
    total_possible_pings = age_counts[age] * num_epochs
    actual_percentage = (total_pings_all_players / total_possible_pings) * 100
    print(f"Leeftijd {age}: {binge_percentages[age]}% doel voor bingedrinken, gemiddeld resultaat: {actual_percentage:.2f}%")

    for player in range(age_counts[age]):
        total_pings = np.sum(ping_tracker[age][player, :])
        min_pings = np.min(ping_tracker[age][player, :])
        max_pings = np.max(ping_tracker[age][player, :])
        print(f"  Speler {player} heeft {total_pings} keer gedronken in {num_epochs} ronden. Min: {min_pings}, Max: {max_pings}")


{12: array([0.        , 0.64984672, 0.        , 0.44476529, 0.24383385]), 13: array([0.        , 0.22457054, 0.        , 0.27389067, 0.35830825]), 14: array([0.3741167 , 0.        , 0.3071135 , 0.        , 0.82751693]), 15: array([0.14565153, 0.        , 0.27070891, 0.0853976 , 0.        ]), 16: array([0.8404953 , 0.18399021, 0.26569265, 0.54068281, 0.        ]), 17: array([0.63197673, 0.11450928, 0.52413663, 0.07739275, 0.        ]), 18: array([0.82488093, 0.75464564, 0.06230109, 0.4308039 , 0.91261661])} 

Leeftijd 12: 0.8% doel voor bingedrinken, gemiddeld resultaat: 0.98%
  Speler 0 heeft 0 keer gedronken in 1000 ronden. Min: 0, Max: 0
  Speler 1 heeft 6 keer gedronken in 1000 ronden. Min: 0, Max: 1
  Speler 2 heeft 0 keer gedronken in 1000 ronden. Min: 0, Max: 0
  Speler 3 heeft 18 keer gedronken in 1000 ronden. Min: 0, Max: 1
  Speler 4 heeft 25 keer gedronken in 1000 ronden. Min: 0, Max: 1
Leeftijd 13: 2.5% doel voor bingedrinken, gemiddeld resultaat: 2.10%
  Speler 0 heeft 0 ke

In [7]:
plt.figure(figsize=(20, 10))
n_ages = len(age_counts)
cmap = cm.get_cmap('RdYlGn_r')
for i, (age, player_count) in enumerate(age_counts.items(), 1):
    ax = plt.subplot(1, n_ages, i)
    pinged_counts = players_pinged[age]
    norm = plt.Normalize(vmin=0, vmax=player_count)
    scatter = ax.scatter(pinged_counts, np.arange(num_epochs), 
                         c=pinged_counts, cmap=cmap, norm=norm, 
                         s=10, alpha=0.7)
    ax.set_title(f"{age} jaar")
    ax.set_xlabel("Aantal agents gepingt")
    ax.set_ylabel("Generaties")
    ax.set_xlim(-1, player_count + 1)
    ax.set_ylim(-50, num_epochs + 50)
    ax.set_yticks(np.arange(0, num_epochs + 1, 100))
    ax.set_xticks([0, 1, 2, 3, 4, 5])
    ax.grid(True)
plt.tight_layout()
# os.makedirs(output_dir, exist_ok=True)
filename = "Aantal agent per eeftijd die willekeurig pingen over 1000 tests.png"
plt.savefig(f"{filename}", dpi=300, bbox_inches='tight')
plt.close()
print(f"opgeslagen")

/var/folders/9x/4jq4r89s0yl_53d6vmg31bcr0000gn/T/ipykernel_1520/3872561746.py:3: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('RdYlGn_r')


opgeslagen


In [34]:
age_counts = {12: 5, 13: 5, 14: 5, 15: 5, 16: 5, 17: 5, 18: 5}
binge_percentages = {12: 0.8, 13: 2.5, 14: 5.5, 15: 13, 16: 37, 17: 50, 18: 88}
num_epochs = 1000
special_event_probability = 0.07
fixed_drinker_counts = {12: 3, 13: 3, 14: 3, 15: 3, 16: 4, 17: 4, 18: 5}

drinkers = {}
ping_tracker = {age: np.zeros((age_counts[age], num_epochs), dtype=int) for age in age_counts}
players_pinged = {age: np.zeros(num_epochs, dtype=int) for age in age_counts}

def gaussian_pings(mean_percentage, num_drinkers, num_epochs):
    mean_pings = mean_percentage / 100 * num_epochs * num_drinkers
    std_dev = max(1, mean_pings * 0.12)
    raw_pings = np.random.normal(mean_pings / num_epochs, std_dev / num_epochs, num_epochs).astype(int)
    raw_pings = np.clip(raw_pings, 0, num_drinkers)
    
    total_pings = raw_pings.sum()
    target_total = int(mean_pings)
    adjustment = target_total - total_pings
    
    while adjustment != 0:
        epoch_indices = np.random.choice(num_epochs, abs(adjustment), replace=True)
        for idx in epoch_indices:
            if adjustment > 0 and raw_pings[idx] < num_drinkers:
                raw_pings[idx] += 1
                adjustment -= 1
            elif adjustment < 0 and raw_pings[idx] > 0:
                raw_pings[idx] -= 1
                adjustment += 1
            if adjustment == 0:
                break
    
    return raw_pings

for age, count in age_counts.items():
    num_drinkers = fixed_drinker_counts[age]
    drinkers[age] = np.zeros(count, dtype=bool)
    drinkers[age][:num_drinkers] = True
    np.random.shuffle(drinkers[age])

for age in age_counts:
    ping_distribution = gaussian_pings(binge_percentages[age], fixed_drinker_counts[age], num_epochs)
    drinker_indices = np.where(drinkers[age])[0]
    for epoch, count in enumerate(ping_distribution):
        if count > 0:
            selected = np.random.choice(drinker_indices, count, replace=False)
            players_pinged[age][epoch] = count
            for player in selected:
                ping_tracker[age][player, epoch] += 1


# Compute normalized ping percentages
normalized_ping_percentage = {}
for age in age_counts:
    total_pings = np.sum(ping_tracker[age])
    num_drinkers = fixed_drinker_counts[age]
    normalized_ping_percentage[age] = (total_pings / (num_epochs * num_drinkers)) * 100

# Print results
for age in age_counts:
    print(f"\nAge {age}: {normalized_ping_percentage[age]:.2f}% pings (target: {binge_percentages[age]}%)")
    for player in range(age_counts[age]):
        total_pings = np.sum(ping_tracker[age][player, :])
        min_pings = np.min(ping_tracker[age][player, :])
        max_pings = np.max(ping_tracker[age][player, :])
        print(f"  Player {player} pinged {total_pings} times over {num_epochs} epochs. Min ping: {min_pings}, Max ping: {max_pings}")



Age 12: 0.80% pings (target: 0.8%)
  Player 0 pinged 8 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 1 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 2 pinged 5 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 3 pinged 11 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 4 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0

Age 13: 2.50% pings (target: 2.5%)
  Player 0 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 1 pinged 27 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 2 pinged 0 times over 1000 epochs. Min ping: 0, Max ping: 0
  Player 3 pinged 28 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 4 pinged 20 times over 1000 epochs. Min ping: 0, Max ping: 1

Age 14: 5.50% pings (target: 5.5%)
  Player 0 pinged 58 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 1 pinged 45 times over 1000 epochs. Min ping: 0, Max ping: 1
  Player 2 pinged 0 times over 1000 epochs. Min ping: 0, M